In [ ]:
# Imports
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams['figure.figsize'] = (15.0, 8.0) # set default size of plots
plt.rcParams['figure.facecolor'] = 'white'

pd.set_option('display.max_rows', None)

matplotlib.rcParams.update({'font.size': 15})

In [ ]:
ANALYSE_WAMR_OFFICIAL = False

In [ ]:
def read_benchmark_data(filename):
    return pd.read_csv(filename, names = ["query_type", "database_type", "number_of_write", "number_of_read", "time"])

def compute_mean(data, query_type, database_type):
    filtered_by_query_type = data.loc[(data["query_type"] == query_type) & (data["database_type"] == database_type)]
    grouped_by_number_of_write = filtered_by_query_type.groupby('number_of_write', as_index=False)['time'].mean()
    return grouped_by_number_of_write

def print_mean_single_values(data, title, epc_limit_size):
    print(title)
    print()
    
    # WASM-mem
    native_mem_wasm_mem = data["native-mem"] / data["wasm-mem"]
    print("native-mem vs wasm-mem:", native_mem_wasm_mem.mean())
    
    # SGX-mem
    native_mem_wasm_sgx_mem_before_epc = data.loc[(data["number_of_write"] < epc_limit_size)]["native-mem"] / data.loc[(data["number_of_write"] < epc_limit_size)]["wasm-sgx-mem"]
    print("native-mem vs wasm-sgx-mem (< EPC):", native_mem_wasm_sgx_mem_before_epc.mean())
    native_mem_wasm_sgx_mem_after_epc = data.loc[(data["number_of_write"] >= epc_limit_size)]["native-mem"] / data.loc[(data["number_of_write"] >= epc_limit_size)]["wasm-sgx-mem"]
    print("native-mem vs wasm-sgx-mem (> EPC):", native_mem_wasm_sgx_mem_after_epc.mean())
    
    # WASM-file
    native_file_wasm_file = data["native-file"] / data["wasm-file"]
    print("native-file vs wasm-file:", native_file_wasm_file.mean())
    
    # SGX-mem
    native_file_wasm_sgx_file_before_epc = data.loc[(data["number_of_write"] < epc_limit_size)]["native-file"] / data.loc[(data["number_of_write"] < epc_limit_size)]["wasm-sgx-file"]
    print("native-file vs wasm-sgx-file (< EPC):", native_file_wasm_sgx_file_before_epc.mean())
    native_file_wasm_sgx_file_after_epc = data.loc[(data["number_of_write"] >= epc_limit_size)]["native-file"] / data.loc[(data["number_of_write"] >= epc_limit_size)]["wasm-sgx-file"]
    print("native-file vs wasm-sgx-file (> EPC):", native_file_wasm_sgx_file_after_epc.mean())
    
    # Values for paper
    print()
    print("Values for paper:")
    wasm_mem_compared_to_native_mem = 1 - native_mem_wasm_mem.mean()
    print(" - Wasm memory is", wasm_mem_compared_to_native_mem * 100,"% slower compared to native memory")
    wasm_file_compared_to_native_mem = 1 - native_file_wasm_file.mean()
    print(" - Wasm file is", wasm_file_compared_to_native_mem * 100,"% slower compared to native file")
    sgx_wasm_mem_compared_to_wasm_mem_before_epc = 1 - (data.loc[(data["number_of_write"] < epc_limit_size)]["wasm-mem"] / data.loc[(data["number_of_write"] < epc_limit_size)]["wasm-sgx-mem"]).mean()
    print(" - SGX Wasm memory is", sgx_wasm_mem_compared_to_wasm_mem_before_epc * 100, "% slower compared to Wasm memory (< EPC)")
    sgx_wasm_mem_compared_to_wasm_mem_after_epc = 1 - (data.loc[(data["number_of_write"] >= epc_limit_size)]["wasm-mem"] / data.loc[(data["number_of_write"] >= epc_limit_size)]["wasm-sgx-mem"]).mean()
    print(" - SGX Wasm memory is", sgx_wasm_mem_compared_to_wasm_mem_after_epc * 100, "% slower compared to Wasm memory (> EPC)")
    sgx_wasm_file_compared_to_wasm_file_before_epc = 1 - (data.loc[(data["number_of_write"] < epc_limit_size)]["wasm-file"] / data.loc[(data["number_of_write"] < epc_limit_size)]["wasm-sgx-file"]).mean()
    print(" - SGX Wasm file is", sgx_wasm_file_compared_to_wasm_file_before_epc * 100, "% slower compared to Wasm file (< EPC)")
    sgx_wasm_file_compared_to_wasm_file_after_epc = 1 - (data.loc[(data["number_of_write"] >= epc_limit_size)]["wasm-file"] / data.loc[(data["number_of_write"] >= epc_limit_size)]["wasm-sgx-file"]).mean()
    print(" - SGX Wasm file is", sgx_wasm_file_compared_to_wasm_file_after_epc * 100, "% slower compared to Wasm file (> EPC)")
    sgx_wasm_memory_compared_to_wasm_file_before_epc = 1 - (data.loc[(data["number_of_write"] < epc_limit_size)]["wasm-file"] / data.loc[(data["number_of_write"] < epc_limit_size)]["wasm-sgx-mem"]).mean()
    print(" - SGX Wasm memory is", sgx_wasm_memory_compared_to_wasm_file_before_epc * 100, "% slower compared to Wasm file (< EPC)")

def merge_benchmark(benchmarks):
    data_merged = benchmarks[0]
    
    for benchmark in benchmarks[1:]:
        data_merged = data_merged.merge(benchmark, on = 'number_of_write')
    
    if (ANALYSE_WAMR_OFFICIAL):
        data_merged.columns = ["number_of_write", "wasm-sgx-file", "wasm-sgx-file-official", "wasm-sgx-mem-official", "wasm-sgx-mem", "wasm-file", "wasm-mem", "native-file", "native-mem"][:len(data_merged.columns)]
    else:
        data_merged.columns = ["number_of_write", "wasm-sgx-file", "wasm-sgx-mem", "wasm-file", "wasm-mem", "native-file", "native-mem"][:len(data_merged.columns)]
    
    return data_merged

def plot_benchmark(data, title, epc_full_size = 0):
    legend_labels = [
        "WebAssembly AoT in SGX (in-memory)",
        "WebAssembly AoT (in-file)",
        "WebAssembly AoT (in-memory)",
        "Native (in-file)",
        "Native (in-memory)",
        "EPC full"
    ]
    
    if (ANALYSE_WAMR_OFFICIAL):
        legend_labels.insert(0, "WebAssembly AoT in SGX official (in-file)")
        legend_labels.insert(1, "WebAssembly AoT in SGX official (in-memory)")
        markers.insert(0, "3")
        markers.insert(1, "1")
        colors.insert(0, "#7f7f7f")
        colors.insert(1, "#e377c2")

    # https://matplotlib.org/3.1.1/api/markers_api.html
    markers = ["D", "P", "v", "s", "o"]
    # https://stackoverflow.com/questions/16006572/plotting-different-colors-in-matplotlib
    colors = ["#9467bd", "#d62728", "#2ca02c", "#ff7f0e", "#1f77b4"]
    
    plot_merged = data.plot(x="number_of_write", title = title)
    plot_merged.set_xlabel("Number of records in the database")
    plot_merged.set_ylabel("time [µs]")
    
    if len(plot_merged.get_lines()) == 6 or len(plot_merged.get_lines()) == 8:
        legend_labels.insert(0, "WebAssembly AoT in SGX (in-file encrypted)")
        markers.insert(0, "*")
        colors.insert(0, "#8c564b")

    for i, line in enumerate(plot_merged.get_lines()):
        #line.set_marker(markers[i])
        line.set_color(colors[i])
     
    # Personnalize the plot
    fmt = "{x:,.0f}"
    tick = matplotlib.ticker.StrMethodFormatter(fmt)
    plot_merged.xaxis.set_major_formatter(tick)
    plot_merged.yaxis.set_major_formatter(tick)
    
    if epc_full_size != 0:
        plt.axvline(x=epc_full_size, linestyle='--', label="epc_full")
    
    plot_merged.legend(legend_labels)

def add_ratio_to_benchmark(data):
    data["ratio_native-mem_wasm-mem"] = data["native-mem"] / data["wasm-mem"]
    data["ratio_native-mem_wasm-sgx-mem"] = data["native-mem"] / data["wasm-sgx-mem"]
    data["ratio-native-file_wasm-file"] = data["native-file"] / data["wasm-file"]
    data["ratio-native-file_sgx-file"] = data["native-file"] / data["wasm-sgx-file"]

In [ ]:
data_native = read_benchmark_data('benchmark-native.csv')
data_wasm = read_benchmark_data('benchmark-wasm.csv')
data_wasm_sgx = read_benchmark_data('benchmark-wasm-sgx.csv')

if (ANALYSE_WAMR_OFFICIAL):
    data_wasm_sgx_official = read_benchmark_data('benchmark-wasm-sgx-official.csv')

In [ ]:
if (ANALYSE_WAMR_OFFICIAL):
    merged_query_seq = merge_benchmark([
        compute_mean(data_wasm_sgx, "qs", 1),
        compute_mean(data_wasm_sgx_official, "qs", 1),
        compute_mean(data_wasm_sgx_official, "qs", 0),
        compute_mean(data_wasm_sgx, "qs", 0),
        compute_mean(data_wasm, "qs", 1),
        compute_mean(data_wasm, "qs", 0),
        compute_mean(data_native, "qs", 1),
        compute_mean(data_native, "qs", 0)
    ])
else:
    merged_query_seq = merge_benchmark([
        compute_mean(data_wasm_sgx, "qs", 1),
        compute_mean(data_wasm_sgx, "qs", 0),
        compute_mean(data_wasm, "qs", 1),
        compute_mean(data_wasm, "qs", 0),
        compute_mean(data_native, "qs", 1),
        compute_mean(data_native, "qs", 0)
    ])

plot_benchmark(merged_query_seq, "time taken to query the number of records of the database sequentially.", 88000)

In [ ]:
if (ANALYSE_WAMR_OFFICIAL):
    merged_query_seq = merge_benchmark([
        compute_mean(data_wasm_sgx_official, "qs", 1),
        compute_mean(data_wasm_sgx_official, "qs", 0),
        compute_mean(data_wasm_sgx, "qs", 0),
        compute_mean(data_wasm, "qs", 1),
        compute_mean(data_wasm, "qs", 0),
        compute_mean(data_native, "qs", 1),
        compute_mean(data_native, "qs", 0)
    ])
else:
    merged_query_seq = merge_benchmark([
        compute_mean(data_wasm_sgx, "qs", 0),
        compute_mean(data_wasm, "qs", 1),
        compute_mean(data_wasm, "qs", 0),
        compute_mean(data_native, "qs", 1),
        compute_mean(data_native, "qs", 0)
    ])

plot_benchmark(merged_query_seq, "time taken to query the number of records of the database sequentially (without SGX in-file).", 88000)

In [ ]:
if (ANALYSE_WAMR_OFFICIAL):
    merged_query_seq = merge_benchmark([
        compute_mean(data_wasm_sgx, "qs", 1),
        compute_mean(data_wasm_sgx_official, "qs", 1),
        compute_mean(data_wasm_sgx_official, "qs", 0),
        compute_mean(data_wasm_sgx, "qs", 0),
        compute_mean(data_wasm, "qs", 1),
        compute_mean(data_wasm, "qs", 0),
        compute_mean(data_native, "qs", 1),
        compute_mean(data_native, "qs", 0)
    ])
else:
    merged_query_seq = merge_benchmark([
        compute_mean(data_wasm_sgx, "qs", 1),
        compute_mean(data_wasm_sgx, "qs", 0),
        compute_mean(data_wasm, "qs", 1),
        compute_mean(data_wasm, "qs", 0),
        compute_mean(data_native, "qs", 1),
        compute_mean(data_native, "qs", 0)
    ])

print_mean_single_values(merged_query_seq, "time taken to query the number of records of the database sequentially", 88000)

In [ ]:
if (ANALYSE_WAMR_OFFICIAL):
    merged_query_seq = merge_benchmark([
        compute_mean(data_wasm_sgx, "qs", 1),
        compute_mean(data_wasm_sgx_official, "qs", 1),
        compute_mean(data_wasm_sgx_official, "qs", 0),
        compute_mean(data_wasm_sgx, "qs", 0),
        compute_mean(data_wasm, "qs", 1),
        compute_mean(data_wasm, "qs", 0),
        compute_mean(data_native, "qs", 1),
        compute_mean(data_native, "qs", 0)
    ])
else:
    merged_query_seq = merge_benchmark([
        compute_mean(data_wasm_sgx, "qs", 1),
        compute_mean(data_wasm_sgx, "qs", 0),
        compute_mean(data_wasm, "qs", 1),
        compute_mean(data_wasm, "qs", 0),
        compute_mean(data_native, "qs", 1),
        compute_mean(data_native, "qs", 0)
    ])

add_ratio_to_benchmark(merged_query_seq)
merged_query_seq

In [ ]:
if (ANALYSE_WAMR_OFFICIAL):
    merged_query_rand = merge_benchmark([
        compute_mean(data_wasm_sgx, "qr", 1),
        compute_mean(data_wasm_sgx_official, "qr", 1),
        compute_mean(data_wasm_sgx_official, "qr", 0),
        compute_mean(data_wasm_sgx, "qr", 0),
        compute_mean(data_wasm, "qr", 1),
        compute_mean(data_wasm, "qr", 0),
        compute_mean(data_native, "qr", 1),
        compute_mean(data_native, "qr", 0)
    ])
else:
    merged_query_rand = merge_benchmark([
        compute_mean(data_wasm_sgx, "qr", 1),
        compute_mean(data_wasm_sgx, "qr", 0),
        compute_mean(data_wasm, "qr", 1),
        compute_mean(data_wasm, "qr", 0),
        compute_mean(data_native, "qr", 1),
        compute_mean(data_native, "qr", 0)
    ])

plot_benchmark(merged_query_rand, "time taken to query the number of records of the database randomly.", 88000)

In [ ]:
if (ANALYSE_WAMR_OFFICIAL):
    merged_query_rand = merge_benchmark([
        compute_mean(data_wasm_sgx_official, "qr", 1),
        compute_mean(data_wasm_sgx_official, "qr", 0),
        compute_mean(data_wasm_sgx, "qr", 0),
        compute_mean(data_wasm, "qr", 1),
        compute_mean(data_wasm, "qr", 0),
        compute_mean(data_native, "qr", 1),
        compute_mean(data_native, "qr", 0),
    ])
else:
    merged_query_rand = merge_benchmark([
        compute_mean(data_wasm_sgx, "qr", 0),
        compute_mean(data_wasm, "qr", 1),
        compute_mean(data_wasm, "qr", 0),
        compute_mean(data_native, "qr", 1),
        compute_mean(data_native, "qr", 0),
    ])

plot_benchmark(merged_query_rand, "time taken to query the number of records of the database randomly (without SGX in-file).", 88000)

In [ ]:
if (ANALYSE_WAMR_OFFICIAL):
    merged_query_rand = merge_benchmark([
        compute_mean(data_wasm_sgx, "qr", 1),
        compute_mean(data_wasm_sgx_official, "qr", 1),
        compute_mean(data_wasm_sgx_official, "qr", 0),
        compute_mean(data_wasm_sgx, "qr", 0),
        compute_mean(data_wasm, "qr", 1),
        compute_mean(data_wasm, "qr", 0),
        compute_mean(data_native, "qr", 1),
        compute_mean(data_native, "qr", 0)
    ])
else:
    merged_query_rand = merge_benchmark([
        compute_mean(data_wasm_sgx, "qr", 1),
        compute_mean(data_wasm_sgx, "qr", 0),
        compute_mean(data_wasm, "qr", 1),
        compute_mean(data_wasm, "qr", 0),
        compute_mean(data_native, "qr", 1),
        compute_mean(data_native, "qr", 0)
    ])

print_mean_single_values(merged_query_rand, "time taken to query the number of records of the databse randomly", 88000)

In [ ]:
if (ANALYSE_WAMR_OFFICIAL):
    merged_query_rand = merge_benchmark([
        compute_mean(data_wasm_sgx, "qr", 1),
        compute_mean(data_wasm_sgx_official, "qr", 1),
        compute_mean(data_wasm_sgx_official, "qr", 0),
        compute_mean(data_wasm_sgx, "qr", 0),
        compute_mean(data_wasm, "qr", 1),
        compute_mean(data_wasm, "qr", 0),
        compute_mean(data_native, "qr", 1),
        compute_mean(data_native, "qr", 0)
    ])
else:
    merged_query_rand = merge_benchmark([
        compute_mean(data_wasm_sgx, "qr", 1),
        compute_mean(data_wasm_sgx, "qr", 0),
        compute_mean(data_wasm, "qr", 1),
        compute_mean(data_wasm, "qr", 0),
        compute_mean(data_native, "qr", 1),
        compute_mean(data_native, "qr", 0)
    ])

add_ratio_to_benchmark(merged_query_rand)
merged_query_rand

In [ ]:
if (ANALYSE_WAMR_OFFICIAL):
    merged_insert = merge_benchmark([
        compute_mean(data_wasm_sgx, "i", 1),
        compute_mean(data_wasm_sgx_official, "i", 1),
        compute_mean(data_wasm_sgx_official, "i", 0),
        compute_mean(data_wasm_sgx, "i", 0),
        compute_mean(data_wasm, "i", 1),
        compute_mean(data_wasm, "i", 0),
        compute_mean(data_native, "i", 1),
        compute_mean(data_native, "i", 0),
    ])
else:
    merged_insert = merge_benchmark([
        compute_mean(data_wasm_sgx, "i", 1),
        compute_mean(data_wasm_sgx, "i", 0),
        compute_mean(data_wasm, "i", 1),
        compute_mean(data_wasm, "i", 0),
        compute_mean(data_native, "i", 1),
        compute_mean(data_native, "i", 0),
    ])

plot_benchmark(merged_insert, "time taken to insert a given amount of rows.", 88000)

In [ ]:
if (ANALYSE_WAMR_OFFICIAL):
    merged_insert = merge_benchmark([
        compute_mean(data_wasm_sgx_official, "i", 1),
        compute_mean(data_wasm_sgx_official, "i", 0),
        compute_mean(data_wasm_sgx, "i", 0),
        compute_mean(data_wasm, "i", 1),
        compute_mean(data_wasm, "i", 0),
        compute_mean(data_native, "i", 1),
        compute_mean(data_native, "i", 0)
    ])
else:
    merged_insert = merge_benchmark([
        compute_mean(data_wasm_sgx, "i", 0),
        compute_mean(data_wasm, "i", 1),
        compute_mean(data_wasm, "i", 0),
        compute_mean(data_native, "i", 1),
        compute_mean(data_native, "i", 0)
    ])

plot_benchmark(merged_insert, "time taken to insert a given amount of rows (without SGX in-file).", 88000)

In [ ]:
if (ANALYSE_WAMR_OFFICIAL):
    merged_insert = merge_benchmark([
        compute_mean(data_wasm_sgx, "i", 1),
        compute_mean(data_wasm_sgx_official, "i", 1),
        compute_mean(data_wasm_sgx_official, "i", 0),
        compute_mean(data_wasm_sgx, "i", 0),
        compute_mean(data_wasm, "i", 1),
        compute_mean(data_wasm, "i", 0),
        compute_mean(data_native, "i", 1),
        compute_mean(data_native, "i", 0)
    ])
else:
    merged_insert = merge_benchmark([
        compute_mean(data_wasm_sgx, "i", 1),
        compute_mean(data_wasm_sgx, "i", 0),
        compute_mean(data_wasm, "i", 1),
        compute_mean(data_wasm, "i", 0),
        compute_mean(data_native, "i", 1),
        compute_mean(data_native, "i", 0)
    ])

print_mean_single_values(merged_insert, "time taken to insert a given amount of rows", 88000)

In [ ]:
if (ANALYSE_WAMR_OFFICIAL):
    merged_insert = merge_benchmark([
        compute_mean(data_wasm_sgx, "i", 1),
        compute_mean(data_wasm_sgx_official, "i", 1),
        compute_mean(data_wasm_sgx_official, "i", 0),
        compute_mean(data_wasm_sgx, "i", 0),
        compute_mean(data_wasm, "i", 1),
        compute_mean(data_wasm, "i", 0),
        compute_mean(data_native, "i", 1),
        compute_mean(data_native, "i", 0)
    ])
else:
    merged_insert = merge_benchmark([
        compute_mean(data_wasm_sgx, "i", 1),
        compute_mean(data_wasm_sgx, "i", 0),
        compute_mean(data_wasm, "i", 1),
        compute_mean(data_wasm, "i", 0),
        compute_mean(data_native, "i", 1),
        compute_mean(data_native, "i", 0)
    ])

add_ratio_to_benchmark(merged_insert)

In [ ]:
#display(compute_mean(data_native, "i", 1))
display(data_native)

In [ ]:
# Export script
#
# Files generated:
# - micro_insert_native_memory_formatted.csv
# - micro_insert_native_file_formatted.csv
# - micro_insert_wasm_memory_formatted.csv
# - micro_insert_wasm_file_formatted.csv
# - micro_insert_wasm_sgx_memory_formatted.csv
# - micro_insert_wasm_sgx_file_formatted.csv
#
# - micro_read_seq_native_memory_formatted.csv
# - micro_read_seq_native_file_formatted.csv
# - micro_read_seq_wasm_memory_formatted.csv
# - micro_read_seq_wasm_file_formatted.csv
# - micro_read_seq_wasm_sgx_memory_formatted.csv
# - micro_read_seq_wasm_sgx_file_formatted.csv
#
# - micro_read_rand_native_memory_formatted.csv
# - micro_read_rand_native_file_formatted.csv
# - micro_read_rand_wasm_memory_formatted.csv
# - micro_read_rand_wasm_file_formatted.csv
# - micro_read_rand_wasm_sgx_memory_formatted.csv
# - micro_read_rand_wasm_sgx_file_formatted.csv
#
# File format: number_of_records, mean, stddev

def export_to_file(dataset, database_type, query_type, filename):
    dataset = dataset \
        .loc[(dataset["query_type"] == query_type) & (dataset["database_type"] == database_type)] \
        .reset_index()
    
    dataset = dataset \
        .drop('number_of_read', axis=1) \
        .drop('database_type', axis=1) \
        .drop('query_type', axis=1)
    
    dataset_mean = dataset.groupby('number_of_write', as_index=False).mean()
    dataset_stddev = dataset.groupby('number_of_write', as_index=False).std()
    
    file = pd.DataFrame(columns = ["database_size", "mean", "stddev"])
    file["database_size"] = (dataset_mean["number_of_write"] / 1000).astype(int) # Convert to MiB
    file["mean"] = dataset_mean["time"] / 1000 / 1000 # Convert to seconds
    file["stddev"] = dataset_stddev["time"]
    
    file.to_csv(filename, index=False)

export_to_file(data_native, 0, 'i', 'micro_insert_native_memory_formatted.csv')
export_to_file(data_native, 1, 'i', 'micro_insert_native_file_formatted.csv')
export_to_file(data_wasm, 0, 'i', 'micro_insert_wasm_memory_formatted.csv')
export_to_file(data_wasm, 1, 'i', 'micro_insert_wasm_file_formatted.csv')
export_to_file(data_wasm_sgx, 0, 'i', 'micro_insert_wasm_sgx_memory_formatted.csv')
export_to_file(data_wasm_sgx, 1, 'i', 'micro_insert_wasm_sgx_file_formatted.csv')

export_to_file(data_native, 0, 'qs', 'micro_read_seq_native_memory_formatted.csv')
export_to_file(data_native, 1, 'qs', 'micro_read_seq_native_file_formatted.csv')
export_to_file(data_wasm, 0, 'qs', 'micro_read_seq_wasm_memory_formatted.csv')
export_to_file(data_wasm, 1, 'qs', 'micro_read_seq_wasm_file_formatted.csv')
export_to_file(data_wasm_sgx, 0, 'qs', 'micro_read_seq_wasm_sgx_memory_formatted.csv')
export_to_file(data_wasm_sgx, 1, 'qs', 'micro_read_seq_wasm_sgx_file_formatted.csv')

export_to_file(data_native, 0, 'qr', 'micro_read_rand_native_memory_formatted.csv')
export_to_file(data_native, 1, 'qr', 'micro_read_rand_native_file_formatted.csv')
export_to_file(data_wasm, 0, 'qr', 'micro_read_rand_wasm_memory_formatted.csv')
export_to_file(data_wasm, 1, 'qr', 'micro_read_rand_wasm_file_formatted.csv')
export_to_file(data_wasm_sgx, 0, 'qr', 'micro_read_rand_wasm_sgx_memory_formatted.csv')
export_to_file(data_wasm_sgx, 1, 'qr', 'micro_read_rand_wasm_sgx_file_formatted.csv')